In [1]:
from models import *
from utils import *
from data_utils import *
import pandas as pd 
pd.set_option("display.precision", 2) # set the precision to 2 decimals

dataset_path = './data/'
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

GPU is available
Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [2]:
aug_type = "window"
strides = [20, 50, 100]
window_sizes = [100, 200, 300, 500, 750, 900]
vote_num = 20 # Make 20 normally
num_folds = 1
num_epochs = 20

print('starting grid search...\n')
for stride in strides:

  print('--------------------------------------------------')
  print('Stride: {}'.format(stride))
  # print('----------')

  test_accs = []
  sub_test_accs = []
  for window_size in window_sizes:
    for k in range(num_folds):
        # indicate hyperparameters here
        model, criterion, optimizer = InitRNN(rnn_type='CNNGRU', dropout=0.6, weight_decay=0.00001)
        # print ('fold {}'.format(k+1))
        
        # Uncomment for just windowing
        X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
        X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

        # Uncomment for cwt
        # X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
        # X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]

        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
        if aug_type != 'window':
            X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num, verbose=False)

    test_acc, sub_test_acc = TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)
    test_accs.append(100 * test_acc)
    sub_test_accs.append(100 * sub_test_acc)
  
  df = pd.DataFrame(list(zip(window_sizes, test_accs, sub_test_accs)), columns = ['Window Size', 'Test Accuracy', 'Subject1 Test Accuracy']) 
  display(df)
  print('\n')



starting grid search...

--------------------------------------------------
Stride: 20
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4244  sub test acc: 0.4600
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.5169  sub test acc: 0.4400
saving best model...
saving best model...
saving best model...
test acc: 0.4808  sub test acc: 0.4200
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.3837  sub test acc: 0.4600
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
t

,Window Size,Test Accuracy,Subject1 Test Accuracy
0,100,42.44,46.0
1,200,51.69,44.0
2,300,48.08,42.0
3,500,38.37,46.0
4,750,30.02,36.0
5,900,34.31,34.0




--------------------------------------------------
Stride: 50
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4221  sub test acc: 0.4000
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4966  sub test acc: 0.5200
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4560  sub test acc: 0.3400
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4492  sub test acc: 0.4800
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
sav

,Window Size,Test Accuracy,Subject1 Test Accuracy
0,100,42.21,40.0
1,200,49.66,52.0
2,300,45.60,34.0
3,500,44.92,48.0
4,750,25.51,28.0
5,900,27.09,32.0




--------------------------------------------------
Stride: 100
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4402  sub test acc: 0.4600
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4786  sub test acc: 0.3800
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.4447  sub test acc: 0.4400
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
saving best model...
test acc: 0.3409  sub test acc: 0.4400
saving best model...
saving best model...
saving best model...
sa

,Window Size,Test Accuracy,Subject1 Test Accuracy
0,100,44.02,46.0
1,200,47.86,38.0
2,300,44.47,44.0
3,500,34.09,44.0
4,750,29.12,28.0
5,900,27.99,26.0
